In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def create_mutation_dataset_with_cnv(mutation_data):
    # Get unique combinations for the mutation columns
    mutation_columns = []
    proteins = mutation_data['Protein_ID'].unique()
    
    # Generate unique mutation columns based on Protein_ID, Position, Reference_AA, and Mutated_AA
    for _, row in mutation_data.iterrows():
        mutation_columns.append(f"{row['Protein_ID']}_{row['Position']}_{row['Reference_AA']}_{row['Mutated_AA']}")

    # Remove duplicates from mutation_columns
    mutation_columns = list(set(mutation_columns))

    # Initialize the dataset (with Yeast_ID and mutation columns)
    yeast_ids = mutation_data['Yeast_ID'].unique()
    mutation_matrix = pd.DataFrame(0, columns=['Yeast_ID'] + mutation_columns + [f"CNV_{protein}" for protein in proteins], index=yeast_ids)
    
    # Set Yeast_ID as the index
    mutation_matrix['Yeast_ID'] = yeast_ids

    # For each yeast, update the mutation and CNV columns
    for _, row in mutation_data.iterrows():
        yeast_id = row['Yeast_ID']
        mutation_column = f"{row['Protein_ID']}_{row['Position']}_{row['Reference_AA']}_{row['Mutated_AA']}"
        
        # Set mutation value to 1 (indicating presence of this mutation in the yeast)
        mutation_matrix.loc[yeast_id, mutation_column] = 1
        
        # Set the CNV for this yeast and protein, if not already set (CNV should be protein-specific)
        if pd.isna(mutation_matrix.loc[yeast_id, f"CNV_{row['Protein_ID']}"]):
            mutation_matrix.loc[yeast_id, f"CNV_{row['Protein_ID']}"] = row['Copy_Number'] if not pd.isna(row['Copy_Number']) else 0  # Default to 0 if NaN

    return mutation_matrix

In [ ]:
merged_data = pd.read_csv("data/merged_mutations_dataset.csv")
X = create_mutation_dataset(merged_data)

/var/folders/hc/f0q04dfj06d1b1f0nks6jk_00000gn/T/ipykernel_17503/2153824515.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[f"{col}_CNV"] = cnv_data[col]
/var/folders/hc/f0q04dfj06d1b1f0nks6jk_00000gn/T/ipykernel_17503/2153824515.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[f"{col}_CNV"] = cnv_data[col]
/var/folders/hc/f0q04dfj06d1b1f0nks6jk_00000gn/T/ipykernel_17503/2153824515.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

In [ ]:
phenotype_data = pd.read_csv("data/phenotype_dataset.csv")
columns_of_interest = ['Standard_name', 'YBL_doublingtime']
filtered_phenotype = phenotype_data[columns_of_interest]
ordered_phenotype = filtered_phenotype.set_index('Standard_name').reindex(X['Yeast_ID']).reset_index()

KeyError: 'Yeast_ID'

In [ ]:

# Charger les datasets
cnv_data = pd.read_csv("copy_number_variation_dataset.csv")
proteome_data = pd.read_csv("proteome_dataset.csv")
phenotype_data = pd.read_csv("phenotype_dataset.csv")

# Joindre les datasets : Copy Number Variations et Proteome
# Fusion basée sur le `Standard_name` ou une autre clé partagée
merged_data = cnv_data.set_index("Standard_name").join(proteome_data.set_index("Protein_ID"), how="inner")
merged_data = merged_data.reset_index()

# Ajouter la cible (YPD_doublingtime) à partir du fichier de phénotypes
merged_data = merged_data.merge(
    phenotype_data[["Standard_name", "YPD_doublingtime"]], 
    left_on="index", 
    right_on="Standard_name", 
    how="inner"
)

# Supprimer les colonnes inutiles
merged_data = merged_data.drop(columns=["Standard_name", "index"])

# Prétraitement des données
# 1. CNV - Normalisation
cnv_features = merged_data.iloc[:, :-2]  # Toutes les colonnes sauf la séquence et YPD_doublingtime
scaler = StandardScaler()
cnv_normalized = scaler.fit_transform(cnv_features)

# 2. Proteome - TF-IDF pour encoder les séquences protéiques
vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(3, 3))  # Tri-grammes pour motifs locaux
proteome_encoded = vectorizer.fit_transform(merged_data["Sequence"])

# Réduction de dimension avec PCA (pour limiter la taille des données après TF-IDF)
pca = PCA(n_components=50)
proteome_reduced = pca.fit_transform(proteome_encoded.toarray())

# Combiner CNV normalisé et protéome réduit
final_data = pd.DataFrame(cnv_normalized).join(pd.DataFrame(proteome_reduced))

# Ajouter la cible (YPD_doublingtime)
final_data["YPD_doublingtime"] = merged_data["YPD_doublingtime"]

# Séparation en ensembles d'entraînement et de test
X = final_data.drop(columns=["YPD_doublingtime"])
y = final_data["YPD_doublingtime"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'pd' is not defined

In [2]:
# Modèle : Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Importance des caractéristiques pour les variations causatives
feature_importances = model.feature_importances_
sorted_indices = feature_importances.argsort()[::-1]
print("Top 10 Feature Importances:")
for idx in sorted_indices[:10]:
    print(f"Feature {idx}: {feature_importances[idx]}")

NameError: name 'RandomForestRegressor' is not defined